In [1]:
import tensorflow as tf
import numpy as np
import librosa as li
import matplotlib.pyplot as plt
import IPython.display as ipd
import time
from preprocessing import Audio
from concurrent.futures import ThreadPoolExecutor

%matplotlib inline
GPU = tf.config.list_physical_devices('GPU')
CPU = tf.config.list_physical_devices('CPU')
DEVICE = GPU[0].name if GPU else CPU[0].name
DEVICE

'/physical_device:GPU:0'

In [2]:
def to_len(y, sec=1.0, sr=44100):
    y, _ = li.effects.trim(y)
    x = tf.keras.preprocessing.sequence.pad_sequences(
        [y],
        maxlen=int(sr * sec),
        padding='post',
        truncating='post',
        dtype='float32'
    )[0]
    return x

In [219]:
def get_audio(path):
    audio = Audio(path)
    augmented = audio.augmented_source()
    with ThreadPoolExecutor(2) as pool:
        data = list(pool.map(to_len, augmented))
    labels = [audio.label] * len(augmented)
    return [data, labels]

In [220]:
def get_labeled_data(nums=1500):
    FALSE_DATA = 'AudioData/0 Данные'
    NOISE = 'AudioData/0 Шум'
    JARVIS = 'AudioData/1 Джарвис'
    DIO = 'AudioData/2 Дио'
    ITAN = 'AudioData/3 Итан'
    LADA = 'AudioData/4 Лада'
    MIRA = 'AudioData/5 Мира'
    
    p = li.util.find_files(FALSE_DATA)
    np.random.shuffle(p)
    PATHS = li.util.find_files(JARVIS) + li.util.find_files(NOISE) + p[:nums]
    
    np.random.shuffle(PATHS)
    
    START_TIME = time.time()
    
    data = list()
    labels = list()
    
    with ThreadPoolExecutor(4) as pool:
        DATA = list(pool.map(get_audio, PATHS))
    
    for d, l in DATA:
        data += d
        labels += l

    data = np.array(data)
    labels = np.array(labels)
    data = np.reshape(data, [*data.shape, 1])
    labels = np.reshape(labels, [*labels.shape, 1])
    print('Loading at {:.2f} seconds'.format(time.time()-START_TIME))
    return (data, labels)

In [221]:
data, labels = get_labeled_data()

Loading at 181.34 seconds


In [200]:
data.shape, labels.shape

((6609, 44100, 1), (6609, 1))

In [197]:
data[0].shape, data[1].shape

((6609, 44100, 1), (6609, 1))

In [230]:
def func(x):
    data = [x] * x
    return [data, data]

with ThreadPoolExecutor(8) as pool:
    y = list(pool.map(func, [1, 2, 3, 4, 5]))

y

[[[1], [1]],
 [[2, 2], [2, 2]],
 [[3, 3, 3], [3, 3, 3]],
 [[4, 4, 4, 4], [4, 4, 4, 4]],
 [[5, 5, 5, 5, 5], [5, 5, 5, 5, 5]]]

In [236]:
tf.constant(y)

ValueError: Can't convert non-rectangular Python sequence to Tensor.